#Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
import gensim
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#Create the DataFrame and extract the necessary columns

In [2]:
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/labeled_data.csv")

print(df.columns)

# Extract the necessary columns
df = df[["tweet", "class"]]

# Convert the columns to lowercase
df['tweet'] = df['tweet'].str.lower()

print(df.head(10))
print(df.shape)

Index(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'class', 'tweet'],
      dtype='object')
                                               tweet  class
0  !!! rt @mayasolovely: as a woman you shouldn't...      2
1  !!!!! rt @mleew17: boy dats cold...tyga dwn ba...      1
2  !!!!!!! rt @urkindofbrand dawg!!!! rt @80sbaby...      1
3  !!!!!!!!! rt @c_g_anderson: @viva_based she lo...      1
4  !!!!!!!!!!!!! rt @shenikaroberts: the shit you...      1
5  !!!!!!!!!!!!!!!!!!"@t_madison_x: the shit just...      1
6  !!!!!!"@__brighterdays: i can not just sit up ...      1
7  !!!!&#8220;@selfiequeenbri: cause i'm tired of...      1
8  " &amp; you might not get ya bitch back &amp; ...      1
9  " @rhythmixx_ :hobbies include: fighting maria...      1
(24783, 2)


#Function to Preprocess the text

In [3]:
import re

def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

#Preprocess the text and extract the output column

In [4]:
tweets = []
sentences = list(df['tweet'])
for sen in sentences:
    tweets.append(preprocess_text(sen))

print(len(tweets))
print(tweets[0])

y = df['class'].to_list()

24783
 rt mayasolovely as woman you shouldn complain about cleaning up your house amp as man you should always take the trash out 


#Import the embeddings

In [5]:
embeddings_url = "/content/gdrive/My Drive/Colab Notebooks/GoogleNews-vectors-negative300.bin"
embeddings = gensim.models.KeyedVectors.load_word2vec_format(embeddings_url, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


#Get the word embeddings of the tweets

In [6]:
# print(embeddings.most_similar('camera', topn = 5))

X = []
Y = []
i = 0

for tweet in tweets:
  # print(i)
  temp = []
  tweet_word_list = tweet.split(" ")
  for word in tweet_word_list:
    try:
      word_vec = embeddings[word] # Check for the presence of the word embedding
      temp.append(word_vec)
    except:
      pass
  if(len(temp) != 0): # To avoid Division by zero error
    X.append(sum(temp)/len(temp))
    Y.append(y[i])
  i+=1

print(len(X))
print(len(X[0]))
print(X[0])

24763
300
[ 0.05579194  0.06608655  0.03765288  0.09530059 -0.07513718  0.03609212
  0.00303214 -0.04977344  0.04857526  0.05521901 -0.01877122 -0.14432925
  0.01272728 -0.00946917 -0.08495367  0.03400239  0.03912644  0.0992141
  0.02508436 -0.06932286 -0.00030118 -0.00521633  0.0624535   0.00100817
 -0.01331148  0.01950053 -0.0820763   0.0930263   0.0386378  -0.03642346
 -0.02860042  0.04071335 -0.06397066 -0.04945156 -0.05108933 -0.04012916
  0.07456752 -0.02220517  0.0221049   0.0944708   0.04629008 -0.09641229
  0.154355   -0.03235226 -0.06574649 -0.03717259 -0.0878325  -0.01153564
  0.00023833 -0.00438045 -0.02029128  0.10078757 -0.02491978  0.02601478
 -0.02058338 -0.00809878 -0.04917544 -0.03198387  0.0820298  -0.02139718
  0.01717413  0.06412978 -0.09806024 -0.08373815  0.0035226  -0.03774007
 -0.05272856  0.07625907 -0.05965006  0.04248628  0.1466762   0.08230591
  0.08786901 -0.00220163 -0.14558338 -0.07003929  0.06448364  0.09080869
  0.01023647  0.07209124 -0.00201707 -0.05

#Form the training and testing data

In [7]:
from sklearn.model_selection import train_test_split

X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(24763, 300)
(24763,)
(17334, 300)
(7429, 300)
(17334,)
(7429,)


#Initialise and run the different models

In [9]:
# Import the necessary models
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# -------------------------------------------------- Gaussian Naive Bayes Model --------------------------------------------------

# Initialize the model
clf_NB = GaussianNB()

print("\n------------------------- Running the GaussianNB Model on the Word Vectors data -------------------------")

# Fit the model on the training data
clf_NB.fit(X_train, y_train)

# Make the predictions
y_pred = clf_NB.predict(X_test)

# Get the overall model performance metrics on the testing set
print("---------- Model Performance Metrics with Gaussian Naive Bayes Model ----------")
print("Accuracy : " + str(accuracy_score(y_test, y_pred, )*100))
print("Precision : " + str(precision_score(y_test, y_pred, average='macro')*100))
print("Recall : " + str(recall_score(y_test, y_pred, average='macro')*100))

# Get the confusion matrix
print(confusion_matrix(y_test, y_pred))


# -------------------------------------------------- Linear SVC Model --------------------------------------------------

# Initialize the model
clf_SVC = SVC(gamma='auto')

print("\n------------------------- Running the SVC Model on Word Vectors data -------------------------")

# Fit the model on the training data
clf_SVC.fit(X_train, y_train)

# Make the predictions
y_pred = clf_SVC.predict(X_test)

# Get the overall model performance metrics on the testing set
print("---------- Model Performance Metrics with SVC Model ----------")
print("Accuracy : " + str(accuracy_score(y_test, y_pred, )*100))
print("Precision : " + str(precision_score(y_test, y_pred, average='macro')*100))
print("Recall : " + str(recall_score(y_test, y_pred, average='macro')*100))

# Get the confusion matrix
print(confusion_matrix(y_test, y_pred))


# -------------------------------------------------- Random Forest Model --------------------------------------------------

# Initialize the model
clf_rf = RandomForestClassifier()

print("\n------------------------- Running the Random Forest Model on the Word Vectors data -------------------------")

# Fit the model on the training data
clf_rf.fit(X_train, y_train)

# Make the predictions
y_pred = clf_rf.predict(X_test)

# Get the overall model performance metrics on the testing set
print("---------- Model Performance Metrics with Random Forest Model ----------")
print("Accuracy : " + str(accuracy_score(y_test, y_pred, )*100))
print("Precision : " + str(precision_score(y_test, y_pred, average='macro')*100))
print("Recall : " + str(recall_score(y_test, y_pred, average='macro')*100))

# Get the confusion matrix
print(confusion_matrix(y_test, y_pred))


------------------------- Running the GaussianNB Model on the Word Vectors data -------------------------
---------- Model Performance Metrics with Gaussian Naive Bayes Model ----------
Accuracy : 69.43060977251312
Precision : 50.579158338810494
Recall : 57.41516609904006
[[ 104  207  132]
 [ 812 4094  842]
 [ 142  136  960]]

------------------------- Running the SVC Model on Word Vectors data -------------------------
---------- Model Performance Metrics with SVC Model ----------
Accuracy : 77.77628213756898
Precision : 55.63611501111501
Recall : 34.20446586464576
[[   0  442    1]
 [   0 5745    3]
 [   0 1205   33]]

------------------------- Running the Random Forest Model on the Word Vectors data -------------------------


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


---------- Model Performance Metrics with Random Forest Model ----------
Accuracy : 84.82972136222911
Precision : 88.8945771605707
Recall : 50.540687224440674
[[   5  388   50]
 [   0 5651   97]
 [   0  592  646]]
